In [15]:
from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt
from pathlib import Path
import numpy as np
from natsort import natsorted


In [16]:
model_path = r"C:\\Users\\AMET\\DocGaid\\yolov8\\runs\\segment\\single_tooth_model_300\\weights\\best.pt"
image_root_path = Path("C:\\Users\\AMET\\DocGaid\\yolov8\\data\\new_single_tooth\\images")

image_paths = list(image_root_path.glob('*.jpg'))
image_paths = natsorted(image_paths)
print(image_paths)


[WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_0.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_1.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_2.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_3.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_4.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_5.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_6.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_7.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_8.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_9.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_10.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117

In [17]:
def rectangle_on_image(image: np.ndarray,
                       box: list,
                       tooth_class: str = None,
                       box_color: tuple = (255, 0, 0),
                       label_color: tuple = (0, 0, 0),
                       thickness: int = 3):
    # Extract coordinates from the box
    x1, y1, x2, y2 = box
    # Draw the rectangle on the image
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), box_color, thickness)
    
    # If a class label is provided, put text on the image
    if tooth_class is not None:
        cv2.putText(image, str(tooth_class), (int(x1) - 10, int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, label_color, 10)
        
    return image

def center_to_xyxy(bbox):
    center_x, center_y, width, height = bbox
    
    x_min = center_x - width / 2
    y_min = center_y - height / 2
    x_max = center_x + width / 2
    y_max = center_y + height / 2

    return [x_min, y_min, x_max, y_max]

def group_paths_by_basename(paths):
    grouped_paths = []
    temp_dict = {}

    for path in paths:
        # Get the base name (part before the underscore)
        base_name = path.stem.split('_')[0]
        # Initialize the list if the base name is not in the temporary dictionary
        if base_name not in temp_dict:
            temp_dict[base_name] = []
        # Append the path to the list
        temp_dict[base_name].append(path)

    # Convert the dictionary to a list of lists
    for paths in temp_dict.values():
        grouped_paths.append(paths)

    return grouped_paths

def get_bbox(polygon):
    polygon = np.array(polygon)
    min_x = np.min(polygon[:, 0])
    min_y = np.min(polygon[:, 1])
    max_x = np.max(polygon[:, 0])
    max_y = np.max(polygon[:, 1])

    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    width = max_x - min_x
    height = max_y - min_y

    return [center_x, center_y, width, height]

def get_class_file_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    bboxes = []
    classes = []
    for line in lines:
        elements = line.split()
        class_id = int(elements[0])
        classes.append(class_id)
        bbox = [float(e) for e in elements[1:]]
        bboxes.append(bbox)
    return np.array(classes), np.array(bboxes)

def denormalize_bbox(bbox, img_shape):
    
    x_center, y_center, width, height = bbox
    x_center *= img_shape[1]
    y_center *= img_shape[0]
    width *= img_shape[1]
    height *= img_shape[0]
    
    return [x_center, y_center, width, height]
    
def get_top_left_coordinates(bbox):
    x_center, y_center, width, height = bbox
    top_left_x = x_center - (width / 2)
    top_left_y = y_center - (height / 2)
    return np.array([top_left_x, top_left_y])

def write_txt_file(txt_file_path, class_id, polygon):
    
    with open(txt_file_path, 'a') as file:
        file.write(f'{class_id}')
        for point in polygon:
            file.write(f" {point}")
        file.write('\n')


grouped_paths = group_paths_by_basename(image_paths)
print(grouped_paths)


[[WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_0.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_1.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_2.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_3.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_4.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_5.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_6.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_7.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_8.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_9.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/117_10.jpg'), WindowsPath('C:/Users/AMET/DocGaid/yolov8/data/new_single_tooth/images/11

In [18]:
import matplotlib.patches as patches

model = YOLO(model_path)
predictions = []

labels_path = Path("C:\\Users\\AMET\\DocGaid\\diploma\\data\\full_teeth\\labels")
images_path = Path("C:\\Users\\AMET\\DocGaid\\diploma\\data\\full_teeth\\images")

for group_path in grouped_paths:
    grouped_predictions = []
    for image_path in group_path:
        txt_file_stem = image_path.stem[:3]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
        results = model.predict([image])  # Pass the image to the predict function
        masks = results[0].masks
        if masks is None:
            grouped_predictions.append([])
            continue
        cropped_polygon = list(masks.xy)
        grouped_predictions.append(cropped_polygon)
        
    full_image_path = (images_path / txt_file_stem).with_suffix('.jpg')  
    txt_file_path = (labels_path / txt_file_stem).with_suffix('.txt')  
    image = cv2.imread(str(full_image_path))
    img_shape = image.shape
    target_classes, target_boxes = get_class_file_data(txt_file_path)
    print(img_shape)
    print(txt_file_path)

    
    
    predictions.append(grouped_predictions)
        # print(type(cropped_polygon))
# predictions


0: 640x256 1 0, 32.0ms
Speed: 2.0ms preprocess, 32.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 256)

0: 640x192 1 0, 25.0ms
Speed: 2.0ms preprocess, 25.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 192)

0: 640x224 1 0, 12.5ms
Speed: 2.0ms preprocess, 12.5ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 224)

0: 640x288 1 0, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 288)

0: 640x320 1 0, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 320)

0: 640x384 1 0, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x224 1 0, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 224)

0: 640x192 1 0, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 192)

0: 640x224 1 0, 10.0ms
Speed: 1

In [19]:
import matplotlib.patches as patches

predictions = []

model = YOLO(model_path)
for group_path in grouped_paths:
    grouped_predictions = []
    for image_path in group_path:
        print(image_path)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
        results = model.predict([image])  # Pass the image to the predict function
        masks = results[0].masks
        if masks is None:
            grouped_predictions.append([])
            continue
        cropped_polygon = list(masks.xy)
        grouped_predictions.append(cropped_polygon)
    predictions.append(grouped_predictions)
        # print(type(cropped_polygon))
# predictions

C:\Users\AMET\DocGaid\yolov8\data\new_single_tooth\images\117_0.jpg

0: 640x256 1 0, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 256)
C:\Users\AMET\DocGaid\yolov8\data\new_single_tooth\images\117_1.jpg

0: 640x192 1 0, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 192)
C:\Users\AMET\DocGaid\yolov8\data\new_single_tooth\images\117_2.jpg

0: 640x224 1 0, 19.0ms
Speed: 1.0ms preprocess, 19.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 224)
C:\Users\AMET\DocGaid\yolov8\data\new_single_tooth\images\117_3.jpg

0: 640x288 1 0, 14.1ms
Speed: 2.0ms preprocess, 14.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 288)
C:\Users\AMET\DocGaid\yolov8\data\new_single_tooth\images\117_4.jpg

0: 640x320 1 0, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 320)
C:\Users\AMET\DocGaid\yolov8\data\new_single_tooth\imag

In [20]:
from itertools import chain

def flatten_list(nested_list):
    return list(chain.from_iterable(chain.from_iterable(nested_list)))

def normalize_segmentations(polygon, img_shape):
    polygon = np.array(polygon)
    polygon[::2] /= img_shape[1]
    polygon[1::2] /= img_shape[0]
    polygon = np.clip(polygon, 0, 1)
  
    return polygon

def denormalize_segmentations(polygon, img_shape):
    polygon = np.array(polygon)
    polygon[::2] *= img_shape[0]
    polygon[1::2] *= img_shape[1]
  
    return polygon


# Example usage:
nested_list = [[[1267.8, 516], [1267.8, 516.78]]]  # Your nested list
flattened_array = flatten_list(nested_list)
print(flattened_array)


[1267.8, 516, 1267.8, 516.78]


In [25]:
labels_path = Path("C:\\Users\\AMET\\DocGaid\\diploma\\data\\full_teeth\\labels")
images_path = Path("C:\\Users\\AMET\\DocGaid\\diploma\\data\\full_teeth\\images")
prediction_labels = Path("C:\\Users\\AMET\\DocGaid\\yolov8\\data\\new_single_tooth\\prediction_labels\\300epoch_ts") 
txt_file_paths = [path for path in labels_path.glob("*.txt") if 117 <= int(path.stem) <= 210]


classes_list = []
target_list = []

for i in range(len(txt_file_paths)):
    # break
    target_classes, target_boxes = get_class_file_data(txt_file_paths[i])
    classes_list.append(target_classes)
    target_list.append(target_boxes)
count = 0

for i in range(len(txt_file_paths)):
    count += 1
    image_path = (images_path/txt_file_paths[i].stem).with_suffix(".jpg")
    print(image_path)
    prediction_label_path = (prediction_labels/txt_file_paths[i].stem).with_suffix(".txt")
    
    image = cv2.imread(str(image_path))
    img_shape = image.shape
    # print(img_shape)
    # break
    polygons = []
    for j in range(len(target_list[i])):
        # print(target_list[i][j])
        # print(j)
        denormalized_bbox = denormalize_bbox(target_list[i][j], img_shape)
        # print(denormalized_bbox)
        add_list = get_top_left_coordinates(denormalized_bbox)

        try:
            # pass
            if type(predictions[i][j]) != list:
                print(f'i {i} j {j}')
                continue
        except IndexError:
            continue
        if isinstance(predictions[i][j], (int, float)):
            print(f'I: {i} J: {j}')
            continue
        if len(predictions[i][j]) > 1:
            predictions[i][j] = [predictions[i][j][0]]
        if predictions[i][j] == []:
            continue
        # print(classes_list[i][j])
        # break
        adjusted_coordinates = predictions[i][j] + add_list
        flattened_array = flatten_list(adjusted_coordinates)
        # print(flattened_array)
        normalized_polygon = normalize_segmentations(flattened_array, img_shape)
        # print(normalized_polygon)
        # break
        write_txt_file(prediction_label_path, classes_list[i][j], normalized_polygon)
        
        # break
    # break
        # denormalized_polygon = denormalize_segmentations(normalized_polygon, img_shape)
        # print(type(adjusted_coordinates[0]))
    # print(denormalized_polygon)
        # break
    # break
    #     polygons.append(adjusted_coordinates)
    #     for polygon in polygons:
    #         pts = np.array(polygon, np.int32)
    #         cv2.polylines(image, [pts], isClosed=True, color=(255, 0, 0), thickness=2)
    #     fig, ax = plt.subplots()
    #     ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  
    # plt.show()

   
    
# print(target_list[0][0])
# print(classes_list)



C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\117.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\118.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\120.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\122.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\123.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\124.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\125.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\126.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\127.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\129.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\130.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\132.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\134.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\135.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\136.jpg
C:\Users\AMET\DocGaid\diploma\data\full_teeth\images\137.jpg
C:\Users\AMET\DocGaid\di

: 